In [1]:
if 'google.colab' in str(get_ipython()):
    !pip uninstall tensorflow -y
    !pip install tensorflow==2.3.1 tensorflow-quantum
    !rm -rf quantum-gans
    !git clone https://github.com/WiktorJ/quantum-gans
    !cd quantum-gans; pip install .

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import math
import random
import tensorflow as tf

import cirq
import numpy as np
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import  get_ground_state_for_g
from qsgenerator.states.simple_state_circuits import build_x_rotation_state
from qsgenerator.states.simple_rotation_generators import get_binary_x_rotation_provider, get_arcsin_x_rotation_provider
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r, get_g_parameters_provider
from qsgenerator.evaluators.circuit_evaluator import CircuitEvaluator

In [38]:
generator_layers = 3
discriminator_layers = 4
data_bus_size = 3

In [39]:
real_phase = False
generic_generator = True
all_layers_labeling = False
full_layer_labeling = False
use_gen_label_qubit = False
use_disc_label_qubit = False

In [40]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(
    generator_layers, 
    discriminator_layers, 
    data_bus_size, 
    all_layers_labeling=all_layers_labeling,
    full_layer_labeling=full_layer_labeling,
    use_gen_label_qubit=use_gen_label_qubit,
    use_disc_label_qubit=use_disc_label_qubit)

In [41]:
if not generic_generator:
    gen, gs = build_ground_state_circuit(qubits=data_qubits, full_parametrization=True)

In [42]:
if real_phase:
    real, real_symbols = build_ground_state_circuit(qubits=data_qubits)
else:
    real, real_symbols = build_x_rotation_state(qubits=data_qubits)

In [43]:
pure_gen = gen.copy()
gen.append([disc])

In [44]:
pure_real = real.copy()
real.append([disc])

In [45]:
print("REAL GROUND STATE")
pure_real

REAL GROUND STATE


(0, 1): ───Rx(r0)───

(0, 2): ───Rx(r1)───

(0, 3): ───Rx(r2)───

In [46]:
print("GENERATOR")
pure_gen

GENERATOR


(0, 1): ───Rx(g0)───Rz(g3)───ZZ──────────────Rx(g8)────Rz(g11)───ZZ────────────────Rx(g16)───Rz(g19)───ZZ────────────────
                             │                                   │                                     │
(0, 2): ───Rx(g1)───Rz(g4)───ZZ^g6───ZZ──────Rx(g9)────Rz(g12)───ZZ^g14───ZZ───────Rx(g17)───Rz(g20)───ZZ^g22───ZZ───────
                                     │                                    │                                     │
(0, 3): ───Rx(g2)───Rz(g5)───────────ZZ^g7───Rx(g10)───Rz(g13)────────────ZZ^g15───Rx(g18)───Rz(g21)────────────ZZ^g23───

In [47]:
print("DISCRIMINATOR")
disc

DISCRIMINATOR


(0, 0): ───Rx(d0)───Rz(d4)───ZZ───────────────Rx(d11)───Rz(d15)───ZZ────────────────Rx(d22)───Rz(d26)───ZZ────────────────Rx(d33)───Rz(d37)───ZZ────────────────
                             │                                    │                                     │                                     │
(0, 1): ───Rx(d1)───Rz(d5)───ZZ^d8───ZZ───────Rx(d12)───Rz(d16)───ZZ^d19───ZZ───────Rx(d23)───Rz(d27)───ZZ^d30───ZZ───────Rx(d34)───Rz(d38)───ZZ^d41───ZZ───────
                                     │                                     │                                     │                                     │
(0, 2): ───Rx(d2)───Rz(d6)───ZZ──────ZZ^d10───Rx(d13)───Rz(d17)───ZZ───────ZZ^d21───Rx(d24)───Rz(d28)───ZZ───────ZZ^d32───Rx(d35)───Rz(d39)───ZZ───────ZZ^d43───
                             │                                    │                                     │                                     │
(0, 3): ───Rx(d3)───Rz(d7)───ZZ^d9────────────Rx(d14)───Rz(d18)───ZZ^d20────────────Rx(d25)───Rz(d29)───ZZ^d31────────────Rx(d36)───Rz(d40)───ZZ^d42────────────

In [48]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [49]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [50]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [92]:
g_values = [0.1, 0.4, 0.25]
# x_rotations = get_binary_x_rotation_provider({0: '100', 1: '011', 2: '101'})
real_values_provider = get_arcsin_x_rotation_provider(g_values, data_bus_size)
#real_values_provider = get_g_parameters_provider()

In [83]:
trainer = Trainer(g_values, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  real_values_provider = real_values_provider,
                  use_analytical_expectation=True)

In [ ]:
epochs = 5
disc_iteration = 20
gen_iteration = 5
snapshot_interval_epochs = 20
results = trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      snapshot_interval_epochs=snapshot_interval_epochs)

In [84]:
def get_gen_for_g(g, gen_weights, gen_provider):
    rad = gen_provider(g)
    return np.append(gen_weights, rad)

def get_states_and_fidelty_for_real(gen_evaluator, 
                                    real_evaluator, 
                                    gen_weights, 
                                    g, 
                                    sized):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights, gen_evaluator.label_value_provider))
    real = real_evaluator.get_state_from_params(real_evaluator.label_value_provider(g))
    return generated, real, cirq.fidelity(generated, real)

def get_states_and_fidelty_for_ground(gen_evaluator, g, gen_weights, size):

    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights, gen_evaluator.label_value_provider), list(range(size)))
    ground = get_ground_state_for_g(g, size)
    return generated, ground, cirq.fidelity(generated, ground)

def compare_generated_for_g(gen_evaluator, g1, g2, gen_weights, size):
    generated1 = gen_evaluator.get_state_from_params(get_gen_for_g(g1, gen_weights, gen_evaluator.label_value_provider), list(range(size)))
    generated2 = gen_evaluator.get_state_from_params(get_gen_for_g(g2, gen_weights, gen_evaluator.label_value_provider), list(range(size)))
    return generated1, generated2, cirq.fidelity(generated1, generated2)

In [85]:
trained_disc_weights = tf.Variable(np.array([ 1.25888796e+01,  1.10409822e+01,  1.27487049e+01,  1.32927475e+01,
       -3.20522385e+01,  2.98508596e+00, -7.54223883e-01,  8.97036648e+00,
        8.98472309e+00, -2.77423954e+00,  8.90891266e+00,  5.72837019e+00,
        6.03105211e+00, -4.64482594e+00, -1.10843427e-01,  7.78598115e-02,
        3.00343895e+00,  3.83781940e-01,  6.08641243e+00,  8.64131927e+00,
       -2.17593918e+01,  1.45857897e+01,  2.36893883e+01,  8.75363445e+00,
       -2.42768993e+01,  1.27688437e+01,  2.53628349e+00,  1.39768391e+01,
        1.40961084e+01, -2.04474068e+01,  4.71392822e+00,  1.66926212e+01,
        1.88311214e+01,  9.17525005e+00,  1.16109962e+01,  1.08004580e+01,
        1.10795708e+01,  5.81477690e+00, -5.50215101e+00,  2.22007637e+01,
        2.25015125e+01,  3.21826210e+01, -1.80058708e+01, -7.85126591e+00,
       -7.77073908e+00,  1.40237570e+01,  3.14071465e+01, -1.07477732e+01,
        1.55852342e+00,  2.63786411e+01,  1.31890945e+01,  7.14759350e+00,
        5.46145630e+00, -1.79730053e+01,  7.40563774e+00,  1.06135674e+01,
       -1.55095673e+00, -2.66580944e+01,  6.51995277e+00, -2.37151980e+00,
        3.51896515e+01,  1.50027342e+01,  1.80733763e-02, -1.75052185e+01,
        9.60706902e+00, -6.06413984e+00,  5.35433817e+00, -7.88707399e+00,
        1.27363043e+01,  1.02446747e+01]), dtype=tf.float32)

trained_gen_weights01 = np.array([
      9.356374740600586,
      3.060741662979126,
      9.562607765197754,
      0.37898221611976624,
      -0.5459064245223999,
      -0.7983298897743225,
      -4.709417819976807,
      1.729026198387146,
      6.080997943878174,
      6.204941749572754,
      6.015588760375977,
      -6.145556926727295,
      -3.898343086242676,
      2.2803266048431396,
      0.8358737230300903,
      -7.79410457611084,
      9.534963607788086,
      -3.393418073654175,
      2.8041188716888428,
      1.7833620309829712,
      -5.981260776519775,
      0.038421839475631714,
      3.6353766918182373,
      -2.8385825157165527
    ])
trained_gen_weights04 = [
      -9.349910736083984,
      -3.137516498565674,
      0.11076110601425171,
      0.1079181432723999,
      -10.369482040405273,
      -1.4755661487579346,
      -7.087393283843994,
      5.457488536834717,
      -2.7765588760375977,
      3.089747667312622,
      12.335768699645996,
      -1.2149827480316162,
      3.585606813430786,
      -8.728550910949707,
      6.772098064422607,
      -2.2341201305389404,
      0.6231995224952698,
      1.1529603004455566,
      4.749046325683594,
      -0.30359596014022827,
      -6.343395233154297,
      3.299123764038086,
      0.01585269533097744,
      8.04002571105957
    ]
gen_symbols = gs 
# gen_symbols = gs
# real_symbols = ('r0', 'r1', 'r2')

In [86]:
gen_evaluator = CircuitEvaluator(pure_gen, gen_symbols, trainer.label_value_provider)
real_evaluator = CircuitEvaluator(pure_real, real_symbols, trainer.real_values_provider)

In [88]:
g = 0.4
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights04, g, data_bus_size)

(array([-0.14824949+0.581017j  ,  0.43242183+0.27149886j,
         0.344239  +0.15211646j,  0.15024966-0.28553918j,
         0.21307297+0.09783034j,  0.1351338 -0.14718074j,
         0.06828734-0.1338553j , -0.11046721-0.06569505j], dtype=complex64),
 array([ 0.6349803 +0.j        ,  0.        -0.47623524j,
         0.        -0.36660606j, -0.27495456+0.j        ,
         0.        -0.27712813j, -0.2078461 +0.j        ,
        -0.16000001+0.j        ,  0.        +0.12j      ], dtype=complex64),
 0.9726657508279892)

In [89]:
g = 0.1
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights01, g, data_bus_size)

(array([-0.6197622 -0.741683j  , -0.15156451+0.08932291j,
        -0.06467123+0.12837054j,  0.02567703-0.00468336j,
        -0.09434147+0.06165439j,  0.00830532+0.01887292j,
        -0.01917351+0.00240218j,  0.00198495+0.00198072j], dtype=complex64),
 array([ 0.92998064+0.j        ,  0.        -0.2924654j ,
         0.        -0.1898315j , -0.05969925+0.j        ,
         0.        -0.09346657j, -0.02939388+0.j        ,
        -0.01907879+0.j        ,  0.        +0.006j     ], dtype=complex64),
 0.972771800733657)

In [108]:
trained_gen_weights025 = [(abs(el1) + abs(el2))/2 for el1,el2 in zip(trained_gen_weights04,trained_gen_weights01)]

In [109]:
trained_gen_weights025

[9.353142738342285,
 3.0991290807724,
 4.836684435606003,
 0.24345017969608307,
 5.457694232463837,
 1.1369480192661285,
 5.8984055519104,
 3.5932573676109314,
 4.428778409957886,
 4.647344708442688,
 9.175678730010986,
 3.6802698373794556,
 3.741974949836731,
 5.504438757896423,
 3.803985893726349,
 5.01411235332489,
 5.079081565141678,
 2.2731891870498657,
 3.7765825986862183,
 1.0434789955615997,
 6.162328004837036,
 1.6687728017568588,
 1.8256146935746074,
 5.4393041133880615]

In [110]:
generated = gen_evaluator.get_state_from_params(get_gen_for_g(0.25, trained_gen_weights025, real_values_provider))
real = real_evaluator.get_state_from_params(real_values_provider(0.25))
generated, real, cirq.fidelity(generated, real)

(array([-0.13393262-0.2619366j ,  0.07937344-0.02939816j,
        -0.26901814-0.2608605j , -0.11164127+0.05950294j,
        -0.08629466+0.2538405j , -0.0899108 -0.05215018j,
         0.32074744+0.70841736j,  0.2456885 -0.04561926j], dtype=complex64),
 array([ 0.80998045+0.j        ,  0.        -0.40815178j,
         0.        -0.30263492j, -0.15249872+0.j        ,
         0.        -0.20913605j, -0.10538433+0.j        ,
        -0.07814   +0.j        ,  0.        +0.039375j  ], dtype=complex64),
 0.1260292243477048)

In [112]:
real025 = real_evaluator.get_state_from_params(real_values_provider(0.25))
real025

array([ 0.80998045+0.j        ,  0.        -0.40815178j,
        0.        -0.30263492j, -0.15249872+0.j        ,
        0.        -0.20913605j, -0.10538433+0.j        ,
       -0.07814   +0.j        ,  0.        +0.039375j  ], dtype=complex64)

In [115]:
real04 = real_evaluator.get_state_from_params(real_values_provider(0.4))
real01 = real_evaluator.get_state_from_params(real_values_provider(0.1))
(real01+real04)/2


array([ 0.7824805 +0.j        ,  0.        -0.3843503j ,
        0.        -0.27821878j, -0.1673269 +0.j        ,
        0.        -0.18529736j, -0.11861999+0.j        ,
       -0.0895394 +0.j        ,  0.        +0.063j     ], dtype=complex64)

In [116]:
cirq.fidelity(real025, (real01+real04)/2)

0.92373884021994

In [ ]:
compare_generated_for_g(gen_evaluator, -0.9, -0.8, trained_gen_weights, data_bus_size)

In [ ]:
g = 0
get_states_and_fidelty_for_ground(gen_evaluator, g, trained_gen_weights, data_bus_size)

In [90]:
pure_gen

(0, 1): ───Rx(g0)───Rz(g3)───ZZ──────────────Rx(g8)────Rz(g11)───ZZ────────────────Rx(g16)───Rz(g19)───ZZ────────────────
                             │                                   │                                     │
(0, 2): ───Rx(g1)───Rz(g4)───ZZ^g6───ZZ──────Rx(g9)────Rz(g12)───ZZ^g14───ZZ───────Rx(g17)───Rz(g20)───ZZ^g22───ZZ───────
                                     │                                    │                                     │
(0, 3): ───Rx(g2)───Rz(g5)───────────ZZ^g7───Rx(g10)───Rz(g13)────────────ZZ^g15───Rx(g18)───Rz(g21)────────────ZZ^g23───